# Building the GBDT here

## Run small data set locally (2-Trees)

In [ ]:
##Let us write spark code here for page rank
import os
import sys

spark_home = os.environ['SPARK_HOME'] = '/Users/maktrix/Dropbox/Berkeley/W261_ML_scale/spark-1.5.1-bin-hadoop2.6'
# spark_home = os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/1.5.0/libexec/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

In [1]:
import os
import sys

spark_home = os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/1.5.0/libexec/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 2.7.10 (default, May 28 2015 17:04:42)
SparkContext available as sc, HiveContext available as sqlContext.


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [1]:
import os
import sys

spark_home = os.environ['SPARK_HOME'] = '/usr/local/spark-1.5.2-bin-hadoop2.6/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.2
      /_/

Using Python version 2.7.6 (default, Sep  9 2014 15:04:36)
SparkContext available as sc, HiveContext available as sqlContext.


In [ ]:
%%writefile GBDT_Model.py
#!/usr/bin/env python
import sys
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from types import *

def parsePoint(point):
    ##exclude the first 13 features always as they are numerical
    return [(i,j)for i,j in enumerate(point.split(',')[14:])]

def createOneHotDict(inputData):
    sampleDistinctFeats = (inputData #.flatMap(filterNumbers)
                         .flatMap(lambda x: x)
                      .distinct())
    return (sampleDistinctFeats
                           .zipWithIndex()
                             .collectAsMap())  

# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDict, numOHEFeats,first13):
    rawFeats=sorted(rawFeats)
    num=[]
    arr=[]
    for j,i in enumerate(first13):
        if i != '':
            num.append(j)
            arr.append(i)
    OHEFeats=[]
    for i in OHEDict.keys():
        if i in rawFeats:
            OHEFeats.append(i)
                  
    return SparseVector(26+13,sorted(num+\
                            [OHEDict[i]+13 for i in OHEFeats]),\
                        arr+[1.0 for i in range(len(OHEFeats))]) #[i for i in first13]+

def parseOHEPoint(point, OHEDict, numOHEFeats):
    featurelist=point.strip().split(',')
    return LabeledPoint(featurelist[0],oneHotEncoding([(i,j) for i,j in enumerate(featurelist[14:]) ],\
                                                      OHEDict, 26+13,[i for i in featurelist[1:14]]))

def maaro(z):
    if z[1]=='':
        return (-1,0)
    else:
        return(z,1)

    
#main GBDT function
def run_GBDT(input_file,output_file,iterations):
    dataRDD=sc.textFile(input_file).map(lambda x: x.replace('\t',','))
    #Now let us create labeled point from data
    dataRDDParsed=dataRDD.map(parsePoint).cache()
    featSet=dataRDDParsed.flatMap(lambda x: x).map(maaro).reduceByKey(lambda a,b: a+b).takeOrdered(26,lambda (k,v): -v)
    #reduceByKey(lambda x,y:x+y).takeOrdered(25,lambda (k,v):-v)
    #print featSet
    #OHEdict=createOneHotDict(dataRDDParsed,featSet)
    OHEdict={}
    for i,x in enumerate(featSet):
#         print i,x
        OHEdict[x[0]]=i
   
    #print oneHotEncoding(dataRDDParsed,OHEdict,numSampleOHEFeats,)
    #Now let us create a dictionary of points
#     weights=[.8,.1,.1]
#     seed=42
#     trainRDD,validateRDD,testRDD=dataRDD.randomSplit(weights,seed)
#     OHETrainData = trainRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
    OHETrainData = dataRDD.map(lambda point: parseOHEPoint(point, OHEdict, 39))
#     print OHETrainData.take(1)
#     print OHETrainData.count()

    model = (GradientBoostedTrees.trainClassifier(OHETrainData, loss = 'logLoss', numIterations=2, 
             categoricalFeaturesInfo={}, learningRate = 0.1, maxDepth = 7, maxBins = 2))
    
    sc.parallelize([model.toDebugString()]).coalesce(1).saveAsTextFile(output_file)  
    
#Execute code    
if __name__== '__main__':
    if len(sys.argv) < 3:
        print >> sys.stderr, "Usage: page_rank <initialize sc ?> <input file> <output file> <damping factor> <number_of_iterations>"
        exit(-1)

    input_file = sys.argv[1]
    output_file = sys.argv[2]
    input_iterations =int(sys.argv[3])
    
    sc = SparkContext(appName="GBDT")
    run_GBDT(input_file,output_file,input_iterations)
    sc.stop()


In [ ]:
#Run first Spark job to generate decision tree rules
!rm -r gbdt_model_output_local
!time /usr/local/Cellar/apache-spark/1.5.0/libexec/bin/spark-submit ./GBDT_Model.py dac_sample.txt gbdt_model_output_local 2
!cat gbdt_model_output_local/part-00000

In [3]:
%%writefile Convert_Python_Rules.py
#!/usr/bin/env python
import sys

def rules_python(input_file, output_file):
    
    spacing = '    '
    
    with open(input_file,'r') as f_in:
        with open(output_file,'w+') as f_out:
            all_lines = []

            for line in f_in:
                # print(model.toDebugString())
    #             model.txt = model.toDebugString().split('\n')

    #             for line in model.txt:
                text = line.split()
                if text == []:
                    pass
                else:
                    all_lines.append(text)

            starting_indent = 1
            if_indents = {0: starting_indent}
            if_num = 0

            for i in range(len(all_lines)):
                if all_lines[i][0] == 'Tree':
                    indents = starting_indent - 1
                    bin_num = 0
#                     tabs = '\t'*starting_indent
                    tabs = spacing*starting_indent
                    tree_num = int(all_lines[i][1].strip(':'))
#                     print tabs + '# Tree ' + str(tree_num)
                    f_out.write(tabs + '# Tree ' + str(tree_num)+'\n')

                elif all_lines[i][0] == 'If':
                    indents = indents + 1
                    if_indents[if_num] = indents
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'if ' + all_lines[i][1] + '[' + all_lines[i][2] + ']' + all_lines[i][3] + all_lines[i][4] + ':'
                    f_out.write(tabs + 'if ' + all_lines[i][1] + '[' + all_lines[i][2] + ']' + all_lines[i][3] + all_lines[i][4] + ':'+'\n')
                    if_num += 1

                elif all_lines[i][0] == 'Else':
                    indents = if_indents[if_num - 1]
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'else:'
                    f_out.write(tabs + 'else:'+'\n')
                    if_num -= 1

                elif all_lines[i][0] == 'Predict:':
                    indents = indents + 1
#                     tabs = '\t'*indents
                    tabs = spacing*indents
#                     print tabs + 'bin_num[' + str(tree_num) + '] = ' + str(bin_num)
                    f_out.write(tabs + 'bin_num[' + str(tree_num) + '] = ' + str(bin_num)+'\n')
                    bin_num += 1
                    indents = indents - 1

if __name__== '__main__':
    rules_python(sys.argv[1],sys.argv[2])
    

Writing Convert_Python_Rules.py


## Run small data set on EMR (2-Trees)

In [ ]:
#copy data file to S3
!aws s3 cp dac_sample.txt s3://ucb-mids-mls-marguerite-oneto/HW14/dac_sample.txt

In [ ]:
%%writefile emr_config_spark_max.json
[
  {
    "Classification": "spark",
    "Properties": {
      "maximizeResourceAllocation": "true"
    }
  }
]


In [ ]:
#create cluster
!aws emr create-cluster --name "14_4_GBDT_test_MO" --release-label emr-4.2.0 --applications Name=Spark --instance-count 3 --use-default-roles --ec2-attributes KeyName=margueriteoneto2 --instance-type m3.xlarge  --enable-debugging --log-uri s3://ucb-mids-mls-marguerite-oneto/HW14/14_4_GBDT --configurations file://./emr_config_spark_max.json

In [ ]:
#upload pyspark script to cluster
!scp -i margueriteoneto2.pem GBDT_Model.py hadoop@ec2-54-153-98-6.us-west-1.compute.amazonaws.com:.

#delete output folder with same name
!aws s3 rm --recursive s3://ucb-mids-mls-marguerite-oneto/HW14/14_4_GBDT/gbdt_model_output_emr

#### Run 1st Spark job on EMR by ssh into the cluster and submit spark job with below commands:
- ssh -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com

- ssh -i margueriteoneto2.pem hadoop@ec2-54-153-98-6.us-west-1.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT_Model.py s3n://hw14/14_4_GBDT/dac_sample.txt s3n://hw14/14_4_GBDT/gbdt_model_output_emr 2

In [ ]:
#KEY STEP: insert the decision tree rules into the Python file
#ALWAYS DO THIS STEP LOCALLY (DON'T TRY TO DO THIS IN THE CLOUD)
!rm -r gbdt_model_output_emr
!aws s3 cp --recursive s3://ucb-mids-mls-marguerite-oneto/HW14/14_4_GBDT/gbdt_model_output_emr gbdt_model_output_emr
!python ./Convert_Python_Rules.py gbdt_model_output_emr/part-00000 gbdt_python_rules.txt
!sed -i '' '/#NEW_RULES_HERE/r gbdt_python_rules.txt' GBDT2_Bins.py

### MO Rewrite of GDBT2_Bins.py With Hashing

In [96]:
%%writefile GBDT2_Bins.py
#!/usr/bin/env python
import sys
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.util import MLUtils
from types import *
import hashlib
import math

#hash like 3 idiots!
def hashLike3Idiots(strToHash,numHashBuckets):
    return int(int(hashlib.md5(strToHash.encode('utf8')).hexdigest(), 16)%numHashBuckets)

def get_hashed_bins(point, numHashBuckets,frequentCategories):
    #input is a tuple
    #1st item is row label,2nd is sparse vector, 3rd item is a tuple (category index array, category value array)
    #[(u'0', 
    #(SparseVector(39, {0: 1.0, 1: 1.0, 2: 5.0, 3: 0.0, 4: 1382.0, 5: 4.0, 6: 15.0, 7: 2.0, 8: 181.0, 9: 1.0, 10: 2.0, 12: 2.0, 13: 1.0, 14: 1.0, 17: 1.0, 19: 1.0, 22: 1.0, 25: 1.0, 28: 1.0, 29: 1.0, 31: 1.0, 32: 1.0}), 
    #([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64], [u'68fd1e64', u'80e26c9b', u'fb936136', u'7b4723c4', u'25c83c98', u'7e0ccccf', u'de7995b8', u'1f89b562', u'a73ee510', u'a8cd5504', u'b2cb9c98', u'37c9c164', u'2824a5f6', u'1adce6ef', u'8ba8b39a', u'891b62e7', u'e5ba7672', u'f54016b9', u'21ddcdc9', u'b1252a9d', u'07b5194c', u'3a171ecb', u'c5c50484', u'e8b83407', u'9727dd16'])))]
    
    feature = SparseVector.toArray(point[1][0])
    
    bin_num = {}
    
#NEW_RULES_HERE
    
    final_features = []
    for i in range(30):
        string = str(i+1) + ":" + str(int(bin_num[i]))
        hashed_bin = int(int(hashlib.md5(string.encode('utf8')).hexdigest(), 16)%numHashBuckets)
        final_features.append(hashed_bin)
    
    #now handle integer, #first 13 items in feature vector is an integer
    for i in range(14):
        #create text like "I1-3"
        key = "I" + str(i)
        featureValue = feature[i]
        #if len(featureValue) == 0: #empty value will be treated as zero
        #    featureValue = 0
        
        if int(featureValue) > 2:
            featureValue = int(math.log(float(featureValue))**2)
        else: 
            featureValue = 'SP'+str(featureValue)
        final_features.append(hashLike3Idiots(key+"-" + str(featureValue),numHashBuckets))
        #final_features.append(key+"-" + str(featureValue))

    #now handle cateogry
    for index, featureValue in enumerate(point[1][1][1]):
        key = "C" + str(index)

        #create text like "C16_913ff151"
        keyValueString = key + "_" + featureValue

        #print keyValueString
        if not keyValueString in frequentCategories: #did this cat happen more than 10 times? 
            keyValueString = key + 'less' #per 3 idiots' source code

        #final_features.append(keyValueString)
        final_features.append(hashLike3Idiots(keyValueString,numHashBuckets))

    label = point[0]

    #print final_features
    
#    new_point = LabeledPoint(point.label, final_features)
    return getFinalLabeledPoint(label, final_features,numHashBuckets)

def parsePoint(point):
    ##exclude the first 13 features always as they are numerical
    return ([(i,j)for i,j in enumerate(point.split(',')[14:])])

def createOneHotDict(inputData):
    sampleDistinctFeats = (inputData #.flatMap(filterNumbers)
                         .flatMap(lambda x: x)
                      .distinct())
    return (sampleDistinctFeats
                           .zipWithIndex()
                             .collectAsMap())  

# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDict, numOHEFeats,first13, last26):
    
    rawFeats=sorted(rawFeats)
    num13=[]
    arr13=[]
    for j,i in enumerate(first13):
        if i != '':
            num13.append(j)
            arr13.append(i)
    num26=[]
    arr26=[]
    for j,i in enumerate(last26):
        if i != '':
            num26.append(j + 39)
            arr26.append(i)
    OHEFeats=[]
    for i in rawFeats:
        if i in OHEDict.keys():
            OHEFeats.append(i)

    return (SparseVector(13+26, \
                        sorted(num13+[OHEDict[i]+13 for i in OHEFeats]),\
                        arr13+[1.0 for i in range(len(OHEFeats))]),
                        (num26,arr26))
            
#    return SparseVector(13+26+26,sorted(num13+\
#                            [OHEDict[i]+13 for i in OHEFeats]+num26),\
#                        arr13+[1.0 for i in range(len(OHEFeats))]+arr26)


def parseOHEPoint(point, OHEDict, numOHEFeats):
    #input
    #(u'0,1,1,5,0,1382,9727dd16, so on... ')
    featurelist=point.strip().split(',')

    new_point = (featurelist[0],oneHotEncoding([(i,j) for i,j in enumerate(featurelist[14:]) ],\
                                                      OHEDict, 13+26+26,[i for i in featurelist[1:14]], \
                                                                        [i for i in featurelist[14:]]))

    return new_point

def maaro(z):
    if z[1]=='':
        return (-1,0)
    else:
        return(z,1)

"""
def parseNonGBDTFeatures(point, frequentCategories,numHashBuckets):
    #format
    #(u'0,1,1,5,0,1382,9727dd16, so on... ', 0) 

    indexCount = 0
    catCount = 0
    features = point[0].split(',')

    modFeatures = []
    
    #print features
    
    for index, featureValue in enumerate(features):
        if index != 0: #ignore first one (label)
            if index <14: #integer
                #create text like "I1-3"
                key = "I" + str(index)
                #print featureValue
                #print index
                if len(featureValue) == 0: #empty value will be treated as zero
                    featureValue = 0
                    
                if int(featureValue) > 2:
                    featureValue = int(math.log(float(featureValue))**2)
                else: 
                    featureValue = 'SP'+str(featureValue)
                modFeatures.append(hashLike3Idiots(key+"-" + str(featureValue),numHashBuckets))
                indexCount +=1
            else:

                key = "C" + str(catCount)
                
                #create text like "C16_913ff151"
                keyValueString = key + "_" + featureValue
                
                #print keyValueString
                
                if not keyValueString in frequentCategories: #did this cat happen more than 10 times? 
                    keyValueString = key + 'less' #per 3 idiots' source code
                    
                modFeatures.append(hashLike3Idiots(keyValueString,numHashBuckets))
                
                catCount +=1
    
    #print indexCount
    #print catCount
    #output format (rowindex, label, features)
    
    #print modFeatures
    label = int(features[0])
    rowIndex = point[1]
    return (rowIndex,(label,modFeatures))
"""

#get Labeled Point based on my arr
def getFinalLabeledPoint(label, myarr,numBuckets):
    
    #remove duplicates
    myarr = sorted(list(set(myarr)))

    keyArray = []
    countArray = []

    for item in myarr:
        keyArray.append(item)
        countArray.append(1)
    #print numBuckets
    #print keyArray
    #print countArray
    #return SparseVector(numBuckets,keyArray,countArray)
    return LabeledPoint(label,SparseVector(numBuckets,keyArray,countArray))

def processFinalJoin(point,numBuckets):
    
    #(x[1][0][0],x[1][0][1] + x[1][1][1]))
    label = point[1][0][0]
    features = point[1][0][1] + point[1][1][1]  
    return getFinalLabeledPoint(label, features,numBuckets)

def run_bins(input_file,output_file, numHashBuckets, frequentCategories_file):
    #repeat of before to generate OHETrainDataBins
    dataRDD=sc.textFile(input_file).map(lambda x: x.replace('\t',','))  #.zipWithIndex()
    #format
    #last integer is an index of the record, we will use it to join result of GBDT with additional Preprocessing B result
    #(u'0,1,1,5,0,1382,9727dd16, so on... ', 0) 
    dataRDD.cache()

#############################    
##### Begin Preprocessing B for integer and categories
#     frequentCategoriesRDD = sc.textFile(frequentCategories_file).collect() #format-- array of text like C14_23425
    
#     freqCategories = sc.broadcast(frequentCategoriesRDD)
#     modifiedIntAndCat = dataRDD.map(lambda x: parseNonGBDTFeatures(x,freqCategories.value, numHashBuckets))
#     #print modifiedIntAndCat.take(1)
#     #format (HASHED of below)
#     #[(rowIndex, (label, ['I1-SP1', 'I2-SP1', 'I3-2', 'I4-SP0', 'I5-52', 'I6-1', 'I7-7', 'I8-SP2', 'I9-27', 'I10-SP1', 'I11-SP2', 'I12-SP0', 'I13-SP2', u'C0_68fd1e64', u'C1_80e26c9b', 'C2less', 'C3less', u'C4_25c83c98', u'C5_7e0ccccf', u'C6_de7995b8', u'C7_1f89b562', u'C8_a73ee510', u'C9_a8cd5504', u'C10_b2cb9c98', 'C11less', u'C12_2824a5f6', u'C13_1adce6ef', u'C14_8ba8b39a', 'C15less', u'C16_e5ba7672', u'C17_f54016b9', u'C18_21ddcdc9', u'C19_b1252a9d', 'C20less', u'C21_', u'C22_3a171ecb', u'C23_c5c50484', u'C24_e8b83407', 'C25less']))]  
##### END Preprocessing B for integer and categories
#############################    

    frequentCategoriesRDD = sc.textFile(frequentCategories_file).collect() #format-- array of text like C14_23425
    freqCategories = sc.broadcast(frequentCategoriesRDD)
    
############################    
##### Begin GBDT processing

    dataRDDParsed=dataRDD.map(parsePoint).cache()
    #format
    #([....(24, u'e8b83407'), (25, u'9727dd16')], 0)
    
    #print dataRDDParsed.take(1)
    featSet=dataRDDParsed.flatMap(lambda x: x).map(maaro) \
        .reduceByKey(lambda a,b: a+b).takeOrdered(26,lambda (k,v): -v)
        
    OHEdict={}
    for i,x in enumerate(featSet):
        OHEdict[x[0]]=i

    OHETrainData = dataRDD.map(lambda point: parseOHEPoint(point, OHEdict, 13+26+26))
    
    #print OHETrainData.take(1)
    #return 
    #OHETrainData should now be in the format:
    #(SparseVector(65, [0, ..., 64], [I0, I1, ..., I12, OHE0, OHE1, ..., OHE25, C0, C1, ..., C25]))
    #print "OHETrain"
    #print OHETrainData.take(1)
        
    #apply the new decision tree
#     OHETrainDataDense = OHETrainData.map(lambda x: SparseVector.toArray(x.features))
#     OHETrainDataBins = OHETrainDataDense.map(lambda x: get_bins(x))
#     OHETrainDataBins.coalesce(1).saveAsTextFile(output_file)


    OHETrainDataBins = OHETrainData.map(lambda point: get_hashed_bins(point, numHashBuckets,freqCategories.value))


#   OHETrainDataBins.saveAsTextFile(output_file)   
    #OHETrainDataBins format:
    #(rowindex, (label, [hashed GBDT assignment 232, 384, 576, 952, 528, 960, 984, 600, 552, 640, 576, 296, 144, 344, 744, 480, 832, 96, 536, 256, 304, 264, 160, 632, 960, 312, 440, 400, 264, 304]))]
##### End GBDT processing    
############################    
    
    
############Now merge set of two features
    #joined = OHETrainDataBins.join(modifiedIntAndCat)
    #after join
    #[(0, ((u'0', [232, 384, 576, 952, 528, 960, 984, 600, 552, 640, 576, 296, 144, 344, 744, 480, 832, 96, 536, 256, 304, 264, 160, 632, 960, 312, 440, 400, 264, 304]),
     #     (0, [480, 720, 688, 856, 392, 24, 184, 480, 432, 696, 16, 368, 352, 368, 368, 160, 608, 392, 680, 768, 936, 432, 424, 112, 840, 264, 536, 152, 80, 440, 360, 640, 600, 520, 304, 128, 592, 952, 536])))]

    #joinedProcessed = joined.map(lambda x: (x[1][0][0],x[1][0][1] + x[1][1][1]))
    #joinedProcessed.cache()
    #print joinedProcessed.take(1)

    #print joinedProcessed.map(lambda x: getFinalLabeledPoint(x[0], x[1], numHashBuckets)).take(1)

    #return 
#     FinalLabeledPoint =  OHETrainDataBins.join(modifiedIntAndCat).map(lambda x: processFinalJoin(x,numHashBuckets))

    #freqCategories.unpersist()
    MLUtils.saveAsLibSVMFile(OHETrainDataBins, output_file) 
    

if __name__== '__main__':
    
    if len(sys.argv) < 4:
        print >> sys.stderr, "Usage: <input file> <output file> <number of hash buckets>  <frequentCategories>"
        exit(-1)

    input_file = sys.argv[1]
    output_file = sys.argv[2]
    numHashBuckets = float(sys.argv[3])
    freqCats = sys.argv[4]
    
    sc = SparkContext(appName="GBDT2")
    run_bins(input_file,output_file, numHashBuckets,freqCats)
    sc.stop()




Overwriting GBDT2_Bins.py


### Rerun Merge of Tree into New GBDT2_Bins.py

In [94]:
#KEY STEP: insert the decision tree rules into the Python file
#ALWAYS DO THIS STEP LOCALLY (DON'T TRY TO DO THIS IN THE CLOUD)
!rm -r gbdt_model_output_emr
!aws s3 cp --recursive s3://ucb-mids-mls-kasaneutsumi/14_4 gbdt_model_output_emr
!python ./Convert_Python_Rules.py gbdt_model_output_emr/part-00000 gbdt_python_rules.txt
!sed -i '' '/#NEW_RULES_HERE/r gbdt_python_rules.txt' GBDT2_Bins.py

download: s3://ucb-mids-mls-kasaneutsumi/14_4/_SUCCESS to gbdt_model_output_emr/_SUCCESS
download: s3://ucb-mids-mls-kasaneutsumi/14_4/part-00000 to gbdt_model_output_emr/part-00000


In [41]:
bleh = [(0, ((u'0', [232, 384, 576, 952, 528, 960, 984, 600, 552, 640, 576, 296, 144, 344, 744, 480, 832, 96, 536, 256, 304, 264, 160, 632, 960, 312, 440, 400, 264, 304]),
         (0, [480, 720, 688, 856, 392, 24, 184, 480, 432, 696, 16, 368, 352, 368, 368, 160, 608, 392, 680, 768, 936, 432, 424, 112, 840, 264, 536, 152, 80, 440, 360, 640, 600, 520, 304, 128, 592, 952, 536])))]

print bleh[0][1][0][0]
print bleh[0][1][0][1] + bleh[0][1][1][1]


for i in range(14):
    print i


0
[232, 384, 576, 952, 528, 960, 984, 600, 552, 640, 576, 296, 144, 344, 744, 480, 832, 96, 536, 256, 304, 264, 160, 632, 960, 312, 440, 400, 264, 304, 480, 720, 688, 856, 392, 24, 184, 480, 432, 696, 16, 368, 352, 368, 368, 160, 608, 392, 680, 768, 936, 432, 424, 112, 840, 264, 536, 152, 80, 440, 360, 640, 600, 520, 304, 128, 592, 952, 536]
0
1
2
3
4
5
6
7
8
9
10
11
12
13


### Run New GBDT2_Bins.py Locally

In [95]:
#Run second Spark job locally to place data into bins
!rm -r gbdt_bins_output_local
!time /usr/local/spark-1.5.2-bin-hadoop2.6/bin/spark-submit \
     ./GBDT2_Bins.py dac_sample.txt gbdt_bins_output_local_betterPerf 1e3 output/    
#!time /usr/local/Cellar/apache-spark/1.5.0/libexec/bin/spark-submit \
#    ./GBDT2_Bins.py dac_sample.txt gbdt_bins_output_local 1e3 output/        
!cat gbdt_bins_output_local/part-00000

rm: gbdt_bins_output_local: No such file or directory
15/12/16 02:08:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/12/16 02:08:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
15/12/16 02:08:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
15/12/16 02:08:47 WARN MetricsSystem: Using default name DAGScheduler for source because spark.app.id is not set.
[Stage 1:>                                                          (0 + 2) / 2]/usr/local/spark-1.5.2-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/shuffle.py:58: UserWarning: Please install psutil to have better support with spilling
/usr/local/spark-1.5.2-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/shuffle.py:58: UserWarning: Please install psutil to have better support with spilling
Traceback (most recent call last):
  File "/Users/kasane/MIDS261/Week14/./GBDT2_Bins.py", line 11794,

In [ ]:
#NEW NEW NEW EMR


nohup time /usr/lib/spark/bin/spark-submit --conf spark.python.worker.reuse=false ./GBDT2_Bins.py s3://criteo-dataset/rawdata/train/ s3://ucb-mids-mls-kasaneutsumi/14_4_BeforeFFM_train 1e6 s3://ucb-mids-mls-kasaneutsumi/14_4_atleast10/ &

### Upload New GBDT2_Bins.py to Cluster

In [ ]:
from pyspark.mllib.linalg import SparseVector
aDense = np.array([0., 3., 0., 4.])
aSparse = SparseVector(len(aDense), [1,3],[3,4])


In [ ]:
#upload second pyspark script to cluster
!scp -i margueriteoneto2.pem GBDT2_Bins.py hadoop@ec2-54-153-4-193.us-west-1.compute.amazonaws.com:.

#delete output folder with same name
!aws s3 rm --recursive s3://ucb-mids-mls-marguerite-oneto/HW14/gbdt_bins_output_emr

#### Run 2nd Spark job on EMR by ssh into the cluster and submit spark job with below commands:
For Arthur:
- ssh -i /Users/maktrix/Dropbox/Berkeley/W261_ML_scale/w261_key2.pem hadoop@ec2-52-35-71-1.us-west-2.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT2_Bins.py s3n://hw14/14_4_GBDT/dac_sample.txt s3n://hw14/14_4_GBDT/gbdt_bins_output_emr

For Marguerite:
- ssh -i margueriteoneto2.pem hadoop@ec2-54-153-4-193.us-west-1.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT2_Bins.py s3://ucb-mids-mls-marguerite-oneto/HW14/dac_sample.txt s3://ucb-mids-mls-marguerite-oneto/HW14/gbdt_bins_output_emr 1e3

- real	1m7.903s
user	0m7.752s
sys	0m0.904s

In [ ]:
#get the bins results
!rm -r gbdt_bins_output_emr
!aws s3 cp --recursive s3://ucb-mids-mls-marguerite-oneto/HW14/14_4_GBDT/gbdt_bins_output_emr gbdt_bins_output_emr
!cat gbdt_bins_output_emr/part-00000

#### Run 2nd Spark job on EMR by ssh into the cluster and submit spark job with below commands using Criteo training data:
For Marguerite:
- ssh -i margueriteoneto2.pem hadoop@ec2-54-153-4-193.us-west-1.compute.amazonaws.com
- time /usr/lib/spark/bin/spark-submit --master yarn-cluster ./GBDT2_Bins.py s3://criteo-dataset/rawdata/train/ s3://ucb-mids-mls-marguerite-oneto/HW14/gbdt_bins_output_criteo 1e3

- real	0m55.073s  
user	0m7.396s  
sys	0m0.984s  